## Лаба 3. Классификация пользователей по интересам в Hive

### Пользователь принадлежит к какой-либо категории, если он посетил хотя бы один из доменов категории хотя бы 10 раз.

In [ ]:
#!/opt/anaconda/envs/bd9/bin/python
# coding: utf-8
import sys
from urllib.parse import urlparse, unquote
import re
def url2domain(url):
    try:
        a = urlparse(unquote(url.strip()))
        if (a.scheme in ['http','https']):
            b = re.search("(?:www\.)?(.*)",a.netloc).group(1)
            if b is not None:
                return str(b).strip()
            else:
                return ''
        else:
            return ''
    except:
        return
    
category1 = [u'cars.ru', u'avto-russia.ru', u'bmwclub.ru']
category2 = ['samara-papa.ru', 'vodvore.net', 'mama51.ru']
category3 = [('sp.krasmama.ru', 'forum.krasmama.ru', 'forum.omskmama.ru']
category4 = ['novayagazeta.ru', 'echo.msk.ru', 'inosmi.ru']
              
for line in sys.stdin:
    line = line.strip().split('\t')
    if len(line)==3:
        uid, ts ,url = line
        if uid != '-' and (url.startswith ('http') or url.startswith ('https')):
            domain=url2domain(url)
            if domain in category1:
                print(int(uid),'\t',domain,'\t','cat1')
            if domain in category2:
                print(int(uid),'\t',domain,'\t','cat2')
            if domain in category3:
                print(int(uid),'\t',domain,'\t','cat3')
            if domain in category4:
                print(int(uid),'\t',domain,'\t','cat4')
#mapper

In [ ]:
#запуск map job
hadoop jar hadoop-streaming.jar -input /labs/lab03data -file ~/maplab3.py
-mapper maplab3.py –output /user/name.surname/lab03

In [ ]:
#Шаг 1 Создание таблицы
CREATE EXTERNAL TABLE lab3_url_logs (
  uid string, 
  domain string, 
  category string) 
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY '\t' 
STORED AS TEXTFILE 
location '/user/name.surname/works';
load data inpath '/user/name.surname/lab03/' into table lab3_url_logs;

In [ ]:
#Шаг 2 Считаем кол-во
select uid,domain,category,count(*) sum
from works 
group by uid, domain, category

In [ ]:
#Шаг 3 Сохранение данных запроса
INSERT OVERWRITE DIRECTORY 'hdfs://bd-master.newprolab.com:8020/user/name.surname/lab03result'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
## верный вариант решения
with T1 as (
select trim(uid) as uid, trim(url) as url, CASE
    WHEN trim(url) in ('cars.ru', 'avto-russia.ru', 'bmwclub.ru') THEN 'Autouser'
    WHEN trim(url) in ('samara-papa.ru', 'vodvore.net', 'mama51.ru') THEN 'Youngparentuser'
    WHEN trim(url) in ('sp.krasmama.ru', 'forum.krasmama.ru', 'forum.omskmama.ru') THEN 'Pregnantuser'
    WHEN trim(url) in ('novayagazeta.ru', 'echo.msk.ru', 'inosmi.ru') THEN 'Politicsuser'
    ELSE 'Other'
  END user_cat, count(*) as cnt_url from lab3_url_logs
group by uid, url
--order by CAST(trim(uid) AS float)
having count(*) >= 10)
SELECT 
 aa.uid,
 NVL(bb.flag_cat1, 0),
 NVL(cc.flag_cat2, 0),
 NVL(dd.flag_cat3, 0),
 NVL(ee.flag_cat4, 0)
FROM (SELECT DISTINCT trim(uid) as uid FROM lab3_url_logs ) aa
LEFT JOIN (SELECT DISTINCT uid, 1 as flag_cat1  FROM T1 WHERE trim(user_cat) = 'Autouser') bb
ON aa.uid = bb.uid 
LEFT JOIN (SELECT DISTINCT uid, 1 as flag_cat2  FROM T1 WHERE trim(user_cat) = 'Youngparentuser') cc
ON aa.uid = cc.uid 
LEFT JOIN (SELECT DISTINCT uid, 1 as flag_cat3  FROM T1 WHERE trim(user_cat) = 'Pregnantuser') dd
ON aa.uid = dd.uid 
LEFT JOIN (SELECT DISTINCT uid, 1 as flag_cat4  FROM T1 WHERE trim(user_cat) = 'Politicsuser') ee
ON aa.uid = ee.uid 
ORDER BY
    CAST(trim(aa.uid) AS float) ASC